In [1]:
from nltk import pos_tag
import nltk
import pandas as pd
import copy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from scipy.sparse import hstack
from tqdm import tqdm
import gensim
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
import numpy as np
from fuzzywuzzy import fuzz

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [41]:
train_df = pd.read_csv('train_cleaned.csv')

In [43]:
[10**-i for i in range(-5, 5)]

[100000, 10000, 1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001]

In [3]:
train_df.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate,cleaned_q1,cleaned_q2
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor koh noor diamond,would happen indian government stole kohinoor ...
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn,internet speed increased hacking dns
3,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder math math divided
4,4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water


In [4]:
pos_tag(nltk.word_tokenize(train_df.loc[0]['cleaned_q1']))

[('step', 'NN'),
 ('step', 'NN'),
 ('guide', 'RB'),
 ('invest', 'JJS'),
 ('share', 'NN'),
 ('market', 'NN'),
 ('india', 'NN')]

In [40]:
train_df[(train_df.cleaned_q1.notnull()) & (train_df.cleaned_q2.notnull()) ]

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate,cleaned_q1,cleaned_q2
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor koh noor diamond,would happen indian government stole kohinoor ...
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn,internet speed increased hacking dns
3,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder math math divided
4,4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water
5,5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,astrology capricorn sun cap moon cap rising say,triple capricorn sun moon ascendant capricorn say
6,6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,buy tiago,keeps childern active far phone video games
7,7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,good geologist,great geologist
8,8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,use instead,use instead
9,9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,motorola company hack charter motorolla dcx,hack motorola dcx free internet


In [5]:
df = copy.deepcopy(train_df[(train_df.cleaned_q1.notnull()) & (train_df.cleaned_q2.notnull()) ])
df.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate,cleaned_q1,cleaned_q2
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor koh noor diamond,would happen indian government stole kohinoor ...
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn,internet speed increased hacking dns
3,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder math math divided
4,4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water


In [6]:
# word to vec using golve
embeddings_index = {}

with open('glove.840B.300d.txt') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding
        
print('Word embeddings: %d' % len(embeddings_index))

Word embeddings: 2196016


In [46]:
embeddings_index['ideas']

array([ -3.53059992e-02,   1.35360003e-01,  -2.05650002e-01,
        -3.31750005e-01,  -2.22110003e-01,   4.27729994e-01,
        -1.96199998e-01,   5.65319993e-02,   2.38020003e-01,
         2.46499991e+00,  -6.53140008e-01,  -2.17969995e-02,
        -1.95680007e-01,   1.52659997e-01,   3.89450014e-01,
        -8.60840008e-02,  -3.85780007e-01,   1.42780006e+00,
        -1.62709996e-01,   6.70540007e-03,   6.07699990e-01,
        -3.06609999e-02,   2.85149992e-01,  -2.77550012e-01,
         1.16559997e-01,   3.24059993e-01,   4.15230006e-01,
         3.86500001e-01,  -4.77039993e-01,  -3.18720013e-01,
         1.05240002e-01,   3.52050006e-01,  -3.85919996e-02,
         1.52089998e-01,   5.42940021e-01,  -6.00100011e-02,
        -4.62779999e-02,  -1.43279999e-01,  -2.95190006e-01,
        -8.21999982e-02,  -4.51370001e-01,   8.01410002e-04,
        -3.89290005e-01,   1.30290002e-01,  -1.16169997e-01,
         4.22450006e-01,  -5.24760000e-02,  -2.90540010e-01,
         3.50919992e-01,

In [7]:
def sent2vec(s):
    words = s.split()
    
    M = []
    
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
            
    M = np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

In [8]:
# question length feature
df['len_q1'] = df.cleaned_q1.apply(lambda x : len(x))
df['len_q2'] = df.cleaned_q2.apply(lambda x : len(x))
df['len_ratio'] = df.len_q1/ df.len_q2
df['diff_len_ratio'] = abs(df.len_q1 - df.len_q2) / (df.len_q1 + df.len_q2)
df.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate,cleaned_q1,cleaned_q2,len_q1,len_q2,len_ratio,diff_len_ratio
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market,41,35,1.171429,0.078947
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor koh noor diamond,would happen indian government stole kohinoor ...,31,67,0.462687,0.367347
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn,internet speed increased hacking dns,44,36,1.222222,0.100000
3,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder math math divided,21,32,0.656250,0.207547
4,4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water,60,29,2.068966,0.348315


In [9]:
# Number of words in question1
df['len_word_q1'] = df.cleaned_q1.apply(lambda x: len(str(x).split()))

# Number of words in question2
df['len_word_q2'] = df.cleaned_q2.apply(lambda x: len(str(x).split()))

# ratio of number of words
df['len_word_ratio'] = df.len_word_q1 / df.len_word_q2

# ratio of difference in these lengths to total length
df['diff_len_word_ratio'] = abs(df.len_word_q1 - df.len_word_q2) / (df.len_word_q1 + df.len_word_q2)

# Number of common words in question1 and question2
df['common_words'] = df.apply(lambda x: len(set(str(x['cleaned_q1']).split()).intersection(set(str(x['cleaned_q2']).split()))), axis=1)

df['common_words_ratio'] = 2* df.common_words / (df.len_word_q1 + df.len_word_q2)

df.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate,cleaned_q1,cleaned_q2,len_q1,len_q2,len_ratio,diff_len_ratio,len_word_q1,len_word_q2,len_word_ratio,diff_len_word_ratio,common_words,common_words_ratio
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market,41,35,1.171429,0.078947,7,6,1.166667,0.076923,5,0.769231
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor koh noor diamond,would happen indian government stole kohinoor ...,31,67,0.462687,0.367347,5,10,0.500000,0.333333,4,0.533333
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn,internet speed increased hacking dns,44,36,1.222222,0.100000,6,5,1.200000,0.090909,2,0.363636
3,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder math math divided,21,32,0.656250,0.207547,3,5,0.600000,0.250000,0,0.000000
4,4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water,60,29,2.068966,0.348315,10,5,2.000000,0.333333,2,0.266667


In [10]:
# TFIDF features
q1_data = df.cleaned_q1.apply(lambda x: str(x))
q2_data = df.cleaned_q2.apply(lambda x: str(x))

q1q2_data = []
q1q2_data.extend(q1_data )
q1q2_data.extend(q2_data )

In [11]:
#initializing vectors for questions 1 and 2
question1_vectors = np.zeros((df.shape[0], 300))

for i, q in tqdm(enumerate(df.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

question2_vectors  = np.zeros((df.shape[0], 300))

for i, q in tqdm(enumerate(df.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

26980it [00:00, 53940.48it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
403902it [00:07, 55098.08it/s]
403902it [00:07, 52498.59it/s]


In [12]:
# cosine distance
df['cosine_distance'] = [np.nan_to_num(cosine(x, y)) for (x, y) in zip(np.nan_to_num(question1_vectors), 
                                                        np.nan_to_num(question2_vectors))]

# cityblock distance
df['cityblock_distance'] = [np.nan_to_num(cityblock(x, y)) for (x, y) in zip(np.nan_to_num(question1_vectors), 
                                                              np.nan_to_num(question2_vectors))]

# jaccard distance
df['jaccard_distance'] = [np.nan_to_num(jaccard(x, y))  for (x, y) in zip(np.nan_to_num(question1_vectors), 
                                                          np.nan_to_num(question2_vectors))]

# canberra distance
df['canberra_distance'] = [np.nan_to_num(canberra(x, y))  for (x, y) in zip(np.nan_to_num(question1_vectors), 
                                                            np.nan_to_num(question2_vectors))]

# euclidean distance
df['euclidean_distance'] = [np.nan_to_num(euclidean(x, y))  for (x, y) in zip(np.nan_to_num(question1_vectors), 
                                                              np.nan_to_num(question2_vectors))]

# minkowski distance
df['minkowski_distance'] = [np.nan_to_num(minkowski(x, y, 3))  for (x, y) in zip(np.nan_to_num(question1_vectors), 
                                                                 np.nan_to_num(question2_vectors))]

# braycurtis distance
df['braycurtis_distance'] = [np.nan_to_num(braycurtis(x, y))  for (x, y) in zip(np.nan_to_num(question1_vectors), 
                                                                np.nan_to_num(question2_vectors))]


/usr/local/lib/python2.7/dist-packages/scipy/spatial/distance.py:505: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))
/usr/local/lib/python2.7/dist-packages/scipy/spatial/distance.py:616: RuntimeWarning: invalid value encountered in double_scalars
  np.double(np.bitwise_or(u != 0, v != 0).sum()))
/usr/local/lib/python2.7/dist-packages/scipy/spatial/distance.py:810: RuntimeWarning: invalid value encountered in double_scalars
  return abs(u - v).sum() / abs(u + v).sum()


In [13]:
# skew of vector for Question 1
df['skew_q1vec'] = [np.nan_to_num(skew(x))  for x in np.nan_to_num(question1_vectors)]

# skew of vector for question 2
df['skew_q2vec'] = [np.nan_to_num(skew(x))  for x in np.nan_to_num(question2_vectors)]

# Kurtosis of vector for Question 1
df['kur_q1vec'] = [np.nan_to_num(kurtosis(x))  for x in np.nan_to_num(question1_vectors)]

# Kurtosis of vector for Question 2
df['kur_q2vec'] = [np.nan_to_num(kurtosis(x))  for x in np.nan_to_num(question2_vectors)]


In [16]:
df.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate,cleaned_q1,cleaned_q2,len_q1,...,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec,fuzz_Qratio
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market,41,...,1.0,77.088724,0.153859,0.070216,0.116865,5.145003,4.783047,55.884817,51.826291,93
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor koh noor diamond,would happen indian government stole kohinoor ...,31,...,1.0,168.039278,0.624010,0.282089,0.463630,3.380478,4.328146,36.672835,62.994071,66
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn,internet speed increased hacking dns,44,...,1.0,144.164826,0.441966,0.206326,0.323486,3.713007,3.534706,42.164445,31.267356,54
3,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder math math divided,21,...,1.0,205.002919,0.850669,0.385899,0.714073,2.506451,5.440028,32.063333,58.885719,35
4,4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water,60,...,1.0,171.876878,0.711627,0.323552,0.488346,1.253517,2.240556,14.765393,29.983146,46


In [17]:
# fuzz features

#Q-ratio
df['fuzz_Qratio'] = df.apply(lambda x: np.nan_to_num(fuzz.QRatio(str(x['question1']), str(x['question2']))), axis=1)

# W-ratio
df['fuzz_WRatio'] = df.apply(lambda x: np.nan_to_num(fuzz.WRatio(str(x['question1']), str(x['question2']))), axis=1)

# Partial ratio
df['fuzz_partial_ratio'] = df.apply(lambda x: np.nan_to_num(fuzz.partial_ratio(str(x['question1']), str(x['question2']))), axis=1)

# Partial token set ratio
df['fuzz_partial_token_set_ratio'] = df.apply(lambda x: np.nan_to_num(fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2']))), axis=1)

# Partial token sort ratio
df['fuzz_partial_token_sort_ratio'] = df.apply(lambda x: np.nan_to_num(fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2']))), axis=1)

# Token set ratio
df['fuzz_token_set_ratio'] = df.apply(lambda x: np.nan_to_num(fuzz.token_set_ratio(str(x['question1']), str(x['question2']))), axis=1)

# Token sort ratio
df['fuzz_token_sort_ratio'] = df.apply(lambda x: np.nan_to_num(fuzz.token_sort_ratio(str(x['question1']), str(x['question2']))), axis=1)

In [38]:
df.count()

Unnamed: 0                       403902
id                               403902
qid1                             403902
qid2                             403902
question1                        403902
question2                        403902
is_duplicate                     403902
cleaned_q1                       403902
cleaned_q2                       403902
len_q1                           403902
len_q2                           403902
len_ratio                        403902
diff_len_ratio                   403902
len_word_q1                      403902
len_word_q2                      403902
len_word_ratio                   403902
diff_len_word_ratio              403902
common_words                     403902
common_words_ratio               403902
cosine_distance                  403902
cityblock_distance               403902
jaccard_distance                 403902
canberra_distance                403902
euclidean_distance               403902
minkowski_distance               403902


In [20]:
#pos tag features
def postag_match_share(q1, q2):
    q1words = {}
    q2words = {}
    for word in q1:
        q1words[word] = 1

    for word in q2:
        q2words[word] = 1

    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2)) / (len(q1words) + len(q2words))
    return R


def get_pos_tag_features(q1,q2):
    
    text = nltk.word_tokenize(q1)
    pq1 = list(map(lambda x: x[1], pos_tag(text)))
    
    text = nltk.word_tokenize(q2)
    pq2 = list(map(lambda x: x[1], pos_tag(text)))
    
    pos_diff = len(set(pq1).difference(set(pq2)))
    pos_symmetric_diff = len(set(pq1).symmetric_difference(set(pq2)))
    
    unique_q1 = len(set(pq1))
    unique_q2 = len(set(pq1))
    if len(pq1) == 0:
        unique_ratio_q1 = 0.0
    else:
        unique_ratio_q1 = len(set(pq1))/float(len(pq1))
        
    if len(pq2) == 0:
        unique_ratio_q2 = 0.0
    else:
        unique_ratio_q2 = len(set(pq2))/float(len(pq2))
        
    match_share = postag_match_share(pq1,pq2)
    
    
    return (pos_diff, 
            pos_symmetric_diff, 
            unique_q1, 
            unique_q2, 
            unique_ratio_q1, 
            unique_ratio_q2, match_share)

In [21]:
pos_diff_list = []
pos_symmetric_diff_list = []
unique_q1_list = []
unique_q2_list = []
unique_ratio_q1_list = []
unique_ratio_q2_list = []
match_share_list = []

for index , row  in tqdm(df[['cleaned_q1','cleaned_q2','is_duplicate']].iterrows()):
    pos_data = get_pos_tag_features(row['cleaned_q1'], row['cleaned_q2'])
    pos_diff_list.append(pos_data[0])
    pos_symmetric_diff_list.append(pos_data[1])
    unique_q1_list.append(pos_data[2])
    unique_q2_list.append(pos_data[3])
    unique_ratio_q1_list.append(pos_data[4])
    unique_ratio_q2_list.append(pos_data[5])
    match_share_list.append(pos_data[6])

403902it [06:23, 1053.12it/s]


In [22]:
df.to_csv('feature_engg_nopos.csv')

In [23]:
len(unique_ratio_q1_list)

403902

In [24]:
pd.Series(pos_diff_list).count()

403902

In [25]:
df.count()

Unnamed: 0                       403902
id                               403902
qid1                             403902
qid2                             403902
question1                        403902
question2                        403902
is_duplicate                     403902
cleaned_q1                       403902
cleaned_q2                       403902
len_q1                           403902
len_q2                           403902
len_ratio                        403902
diff_len_ratio                   403902
len_word_q1                      403902
len_word_q2                      403902
len_word_ratio                   403902
diff_len_word_ratio              403902
common_words                     403902
common_words_ratio               403902
cosine_distance                  403902
cityblock_distance               403902
jaccard_distance                 403902
canberra_distance                403902
euclidean_distance               403902
minkowski_distance               403902


In [26]:
df2 = copy.deepcopy(df)

In [29]:
df2['pos_diff'] = map(lambda x: x, pos_diff_list)



Unnamed: 0                       403902
id                               403902
qid1                             403902
qid2                             403902
question1                        403902
question2                        403902
is_duplicate                     403902
cleaned_q1                       403902
cleaned_q2                       403902
len_q1                           403902
len_q2                           403902
len_ratio                        403902
diff_len_ratio                   403902
len_word_q1                      403902
len_word_q2                      403902
len_word_ratio                   403902
diff_len_word_ratio              403902
common_words                     403902
common_words_ratio               403902
cosine_distance                  403902
cityblock_distance               403902
jaccard_distance                 403902
canberra_distance                403902
euclidean_distance               403902
minkowski_distance               403902


In [30]:
df['pos_diff'] = map(lambda x: x, pos_diff_list)
df['pos_symmetric_diff'] = map(lambda x: x, pos_symmetric_diff_list)
df['unique_q1'] = map(lambda x: x, unique_q1_list)
df['unique_q2'] = map(lambda x: x, unique_q2_list)
df['unique_ratio_q1'] = map(lambda x: x, unique_ratio_q1_list)
df['unique_ratio_q2'] = map(lambda x: x, unique_ratio_q2_list)
df['match_share'] = map(lambda x: x, match_share_list)

In [31]:
df.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate,cleaned_q1,cleaned_q2,len_q1,...,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio,pos_diff,pos_symmetric_diff,unique_q1,unique_q2,unique_ratio_q1,unique_ratio_q2,match_share
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market,41,...,89,100,93,0,0,3,3,0.428571,0.5,1
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor koh noor diamond,would happen indian government stole kohinoor ...,31,...,75,86,63,0,4,2,2,0.400000,0.6,0
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn,internet speed increased hacking dns,44,...,71,66,66,0,1,3,3,0.500000,0.8,0
3,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder math math divided,21,...,38,36,36,1,3,2,2,0.666667,0.6,0
4,4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water,60,...,63,67,47,4,6,5,5,0.500000,0.6,0


In [32]:
df[df.isnull().any(axis=1)]

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate,cleaned_q1,cleaned_q2,len_q1,...,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio,pos_diff,pos_symmetric_diff,unique_q1,unique_q2,unique_ratio_q1,unique_ratio_q2,match_share


In [33]:
df.to_csv('feature_engg.csv')

In [74]:
pq1 = list(map(lambda x: x[1], pos_tag(nltk.word_tokenize(df.loc[0]['cleaned_q1']))))
pq2 = list(map(lambda x: x[1], pos_tag(nltk.word_tokenize(df.loc[0]['cleaned_q2']))))

In [75]:
len(set(pq1).difference(set(pq2)))

0

In [35]:
!pwd

/notebooks/sharedfolder


In [34]:
df.loc[1]

Unnamed: 0                                                                       1
id                                                                               1
qid1                                                                             3
qid2                                                                             4
question1                        What is the story of Kohinoor (Koh-i-Noor) Dia...
question2                        What would happen if the Indian government sto...
is_duplicate                                                                     0
cleaned_q1                                         story kohinoor koh noor diamond
cleaned_q2                       would happen indian government stole kohinoor ...
len_q1                                                                          31
len_q2                                                                          67
len_ratio                                                                 0.462687
diff

In [53]:
df.loc[193246]

Unnamed: 0                                                                  193246
id                                                                          193246
qid1                                                                        237200
qid2                                                                        279869
question1                        What was it like to be in the game industry du...
question2                                                                [removed]
is_duplicate                                                                     0
cleaned_q1                       like game industry moral panic video games cau...
cleaned_q2                                                                 removed
len_q1                                                                         116
len_q2                                                                           7
len_ratio                                                                  16.5714
diff

In [42]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df= 0 , stop_words='english')

tf_feat = tfidf.fit_transform(q1_data)